In [1]:
import pandas as pd
import random
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dir = r'/content/drive/MyDrive/CSE4214/Project/prothom_alo_punctuation_balanced.txt'

In [4]:
my_file = open(dir, "r")
data = my_file.read()
sentences = data.split("\n")
print(sentences[0])
my_file.close()

print(sentences[0])

এই মুহূর্তে এই ধ্বনিতেই উচ্চকিত ফুটবল বিশ্ব।
এই মুহূর্তে এই ধ্বনিতেই উচ্চকিত ফুটবল বিশ্ব।


In [5]:
last_characters = [sentence[-1] for sentence in sentences]

bengali_punctuation_dict = {"।": "", ",": "", "?": "", "!": "", ";": "", ":": "", "—": " "}

translator = str.maketrans(bengali_punctuation_dict)

cleaned_sentences = [sentence.translate(translator) for sentence in sentences]

labels = []
for char in last_characters:
    if char == '।':
        labels.append('0')
    elif char == '?':
        labels.append('1')
    elif char == '!':
        labels.append('2')

data = {'Sentence': cleaned_sentences, 'Label': labels}

df = pd.DataFrame(data)

df.head(10)


,Sentence,Label
0,এই মুহূর্তে এই ধ্বনিতেই উচ্চকিত ফুটবল বিশ্ব,0
1,এক আসর থেকে আর কি-ই বা চাওয়া থাকতে পারে ২১ বছ...,2
2,কথাটা নিজের মুখেই বলেছেন নেইমার,0
3,তিনি বলেন ‘যারা সেনাবাহিনীর বিরুদ্ধে অপপ্রচার ...,1
4,প্রায় ৪৫ গজ দূর থেকে নেওয়া শটে করেছেন লক্ষ্য...,2
5,গত ক্লাব বিশ্বকাপের পর আবার একসঙ্গে মাঠে দেখা ...,0
6,৪৫ গজ দূর থেকে নেইমার হঠাত্ এমন শট নিলেন কেউ ক...,2
7,উরুগুইয়ান গোলরক্ষক ফার্নান্দো মুসলেরা চেষ্টা ...,0
8,রূপকথার গল্প তো মানুষ এমনিতেই বেশি পছন্দ করে,2
9,সূত্র ওয়েবসাইট,0


In [6]:
df['word_count'] = df['Sentence'].apply(lambda x: len(x.split()))
df = df.loc[df['word_count'] <= 50]
df.head(10)

,Sentence,Label,word_count
0,এই মুহূর্তে এই ধ্বনিতেই উচ্চকিত ফুটবল বিশ্ব,0,7
1,এক আসর থেকে আর কি-ই বা চাওয়া থাকতে পারে ২১ বছ...,2,13
2,কথাটা নিজের মুখেই বলেছেন নেইমার,0,5
3,তিনি বলেন ‘যারা সেনাবাহিনীর বিরুদ্ধে অপপ্রচার ...,1,15
4,প্রায় ৪৫ গজ দূর থেকে নেওয়া শটে করেছেন লক্ষ্য...,2,10
5,গত ক্লাব বিশ্বকাপের পর আবার একসঙ্গে মাঠে দেখা ...,0,13
6,৪৫ গজ দূর থেকে নেইমার হঠাত্ এমন শট নিলেন কেউ ক...,2,17
7,উরুগুইয়ান গোলরক্ষক ফার্নান্দো মুসলেরা চেষ্টা ...,0,10
8,রূপকথার গল্প তো মানুষ এমনিতেই বেশি পছন্দ করে,2,8
9,সূত্র ওয়েবসাইট,0,2


In [7]:
df['Label'].nunique()

3

In [8]:
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import SpatialDropout1D
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from sklearn.feature_selection import RFE
from keras.layers import Conv1D
from keras.layers import MaxPooling1D

In [10]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 100
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 100
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~।', lower=False)
tokenizer.fit_on_texts(df.Sentence.values)
word_index = tokenizer.word_index
vocab_size = len(word_index)
print('Found %s unique tokens.' % vocab_size)


Found 218955 unique tokens.


In [11]:
X = tokenizer.texts_to_sequences(df.Sentence.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (448285, 100)


In [12]:
Y = pd.get_dummies(df.Label).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (448285, 3)


In [13]:
train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size=.20, stratify=Y, shuffle=True)

In [14]:
# Note this is the 100 dimension version of GloVe from Stanford
# It was unzipped and hosted on a  site to make this notebook easier
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, EMBEDDING_DIM));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;


--2023-08-16 21:45:15--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.192.128, 142.250.152.128, 209.85.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.192.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-08-16 21:45:15 ERROR 404: Not Found.



In [15]:
print(len(embeddings_matrix))

218956


In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size+1, EMBEDDING_DIM, input_length=X.shape[1], weights=[embeddings_matrix]))
# experiment with combining different types, such as convolutions and LSTMs
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv1D(100, 5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size=4))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          21895600  
                                                                 
 dropout (Dropout)           (None, 100, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 96, 100)           50100     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 100)          0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                        

In [17]:
epochs = 5
batch_size = 16

history = model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)], verbose=2)


Epoch 1/5


KeyboardInterrupt: ignored

In [ ]:
accr = model.evaluate(test_features,test_labels)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


2802/2802 [==============================] - 10s 4ms/step - loss: 1.0987 - accuracy: 0.3321
Test set
  Loss: 1.099
  Accuracy: 0.332
